In [2]:
import polars as pl
import numpy as np, pandas as pd
print(
    f"Polars version = {pl.__version__}, Numpy version = {np.__version__}"
)
%load_ext autoreload
%autoreload 2
from src import data_transformation

Polars version = 1.32.2, Numpy version = 1.26.4


# 数据处理

In [3]:
from pathlib import Path
INPUTS = Path('./data/inputs')
INTERMEDIATES = Path('./data/intermediates')
OUTPUTS = Path('./data/outputs')

In [ ]:
## 
datasets = data_transformation.load_inputs(INPUTS)
intermediate_datasets_pth = {
    "merged_reserve": INTERMEDIATES/"merged_reserve.csv",
    "date_info": INTERMEDIATES/"date_info.csv",
    "merged_store_info": INTERMEDIATES/"merged_store_info.csv",
    "train_visit_data": INTERMEDIATES/"train_visit_data.csv",
    "test_visit_data": INTERMEDIATES/"test_visit_data.csv"
}

if not intermediate_datasets_pth["merged_reserve"].exists():
    merged_reserve = data_transformation.merge_reservation(**datasets)
    merged_reserve.sink_csv(INTERMEDIATES/"merged_reserve.csv")

if not intermediate_datasets_pth["date_info"].exists():
    datasets["date_info"].sink_csv(INTERMEDIATES/"date_info.csv")

if not intermediate_datasets_pth["merged_store_info"].exists():
    datasets["air_store_info"].sink_csv(INTERMEDIATES/"merged_store_info.csv")

if not intermediate_datasets_pth["train_visit_data"].exists():
    datasets["air_visit_data"].filter(pl.col("visit_date") <= pl.lit("2017-03-10")).sink_csv(INTERMEDIATES/"train_visit_data.csv")
if not intermediate_datasets_pth["test_visit_data"].exists():
    datasets["air_visit_data"].filter(pl.col("visit_date") > pl.lit("2017-03-10")).sink_csv(INTERMEDIATES/"test_visit_data.csv")

In [13]:
output_datasets_pth = {
    "hpg_train": OUTPUTS/"hpg_train.parquet",
    "hpg_test": OUTPUTS/"hpg_test.parquet",
    "air_train": OUTPUTS/"air_train.parquet",
    "air_test": OUTPUTS/"air_test.parquet",
}

if any([not pth.exists() for pth in output_datasets_pth.values()]):
    train, test = data_transformation.add_features_pipeline(
        intermediate_datasets_pth["train_visit_data"],
        intermediate_datasets_pth["test_visit_data"],
        intermediate_datasets_pth["merged_reserve"],
        intermediate_datasets_pth["date_info"],
        intermediate_datasets_pth["merged_store_info"]
    )
    hpg_list = pd.read_csv(INPUTS/"store_id_relation.csv")["air_store_id"].tolist()

    train.query("air_store_id in @hpg_list").to_parquet(output_datasets_pth["hpg_train"])
    train.query("air_store_id not in @hpg_list").to_parquet(output_datasets_pth["air_train"])
    test.query("air_store_id in @hpg_list").to_parquet(output_datasets_pth["hpg_test"])
    test.query("air_store_id not in @hpg_list").to_parquet(output_datasets_pth["air_test"])

# 模型训练

In [ ]:
# train_matrix = clf.Dataset(tr_x, label=tr_y,weight=weight_train["day_gap"])
# test_matrix = clf.Dataset(te_x, label=te_y,weight=weight_test["day_gap"])

# params = {
#     # Feiyang: 10. 把 7 改成了 8
#     'num_leaves': 2 ** 8 - 1,
#     'objective': 'regression_l2',
#     # Feiyang: 11. 把 8 改成了 9
#     'max_depth': 9,
#     'min_data_in_leaf': 50,
#     # Feiyang: 12. 把 0.01 改成了 0.007 并同时改了下面的 Num_round 和 early_stopping_rounds
#     'learning_rate': 0.007,
#     'feature_fraction': 0.6,
#     # Feiyang: 13. 把 0.75 改成了 0.8
#     'bagging_fraction': 0.8,
#     'bagging_freq': 1,
#     'metric': 'rmse',
#     'num_threads': 4,
#     'seed': 2018,
# }

# num_round = 6000
# early_stopping_rounds = 500
# if test_matrix:
#     model = clf.train(params, train_matrix,num_round,valid_sets=test_matrix,
#                         early_stopping_rounds=early_stopping_rounds
#                         )
#     pre= model.predict(te_x,num_iteration=model.best_iteration).reshape((te_x.shape[0],1))

,air_store_id,visit_date,day_gap,visitors,weekday,day,days_to_side,air_genre_name,air_area_name,latitude,...,air_genre_name_weekday_min_420,air_genre_name_weekday_count_420,air_area_name_holiday_kermed_420,air_area_name_holiday_max_420,air_area_name_holiday_min_420,air_area_name_holiday_count_420,air_area_genre_name_holiday_kermed_420,air_area_genre_name_holiday_max_420,air_area_genre_name_holiday_min_420,air_area_genre_name_holiday_count_420
0,air_ba937bf13d40fb24,2017-02-04,0,3.433987,5,0,4,4,62,35.658068,...,0.693147,4026.0,2.694135,4.867534,0.693147,2790.0,2.766056,4.276666,0.693147,522.0
1,air_ba937bf13d40fb24,2017-02-05,1,1.386294,6,0,5,4,62,35.658068,...,0.693147,3375.0,2.694135,4.867534,0.693147,2790.0,2.766056,4.276666,0.693147,522.0
2,air_ba937bf13d40fb24,2017-02-06,2,2.484907,0,0,6,4,62,35.658068,...,0.693147,3419.0,2.741566,5.723585,0.693147,8522.0,2.893790,4.382027,0.693147,1420.0
3,air_ba937bf13d40fb24,2017-02-07,3,2.197225,1,0,7,4,62,35.658068,...,0.693147,3822.0,2.741566,5.723585,0.693147,8522.0,2.893790,4.382027,0.693147,1420.0
4,air_ba937bf13d40fb24,2017-02-08,4,3.135494,2,1,8,4,62,35.658068,...,0.693147,3923.0,2.741566,5.723585,0.693147,8522.0,2.893790,4.382027,0.693147,1420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9165,air_6c952e3c6e590945,2016-02-15,30,2.639057,0,1,13,13,32,38.269076,...,1.098612,11.0,2.830974,4.060443,0.693147,77.0,2.944439,2.944439,2.944439,1.0
9166,air_6c952e3c6e590945,2016-02-16,31,3.332205,1,1,12,13,32,38.269076,...,2.397895,10.0,2.830974,4.060443,0.693147,77.0,2.944439,2.944439,2.944439,1.0
9167,air_6c952e3c6e590945,2016-02-17,32,2.995732,2,1,11,13,32,38.269076,...,2.079442,12.0,2.830974,4.060443,0.693147,77.0,2.944439,2.944439,2.944439,1.0
9168,air_6c952e3c6e590945,2016-02-18,33,2.944439,3,1,10,13,32,38.269076,...,1.945910,12.0,2.830974,4.060443,0.693147,77.0,2.944439,2.944439,2.944439,1.0
